In [1]:
import sqlite3
import pandas as pd

### Section I 

In [2]:
conn = sqlite3.connect('owners.db')
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('owners',)]

In [3]:
pd.read_sql_query("SELECT * FROM owners;",conn)

,Name,Phone,Address,ID
0,Josh Daniell,3132879573,1440 G st.,1
1,Alison Peebles Madigan,2147098190,None,2
2,Avi Flombaum,4698780125,1776 New York Ave,3
3,Justin Bieber,4103810987,None,4


In [4]:
c.execute("ATTACH DATABASE 'cats.db' as cats_db")
c.execute("ATTACH DATABASE 'dogs.db' as dogs_db")

In [6]:
pd.read_sql_query("""SELECT owners.Name,
                            cats.Name catName,
                            dogs.Name dogName
                     FROM owners
                     JOIN dogs on owners.ID = dogs.owner_ID
                     JOIN cats on owners.ID = cats.owner_ID
                     WHERE owners.Name = 'Justin Bieber'
                ;""",conn)

,Name,catName,dogName
0,Justin Bieber,Bell,Bobo
1,Justin Bieber,Precious,Bobo


### Section II - 2 possible solutions 

#### a.

In [ ]:
c.execute("""SELECT house_number, count((house_number)) as count FROM 
(SELECT * FROM dogs
UNION 
SELECT * FROM cats) AS AB
WHERE house_number IS NOT '' 
GROUP BY house_number
ORDER BY count DESC; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

# b.

In [ ]:
c.execute("""WITH dogs_count AS (SELECT house_number, count(name) AS num_dogs
FROM dogs 
GROUP BY house_number), cats_count AS 
(SELECT house_number, count(name) AS num_cats
FROM cats 
GROUP BY house_number)
SELECT dogs_count.house_number, num_dogs + num_cats as sum_pets FROM dogs_count
LEFT JOIN cats_count on dogs_count.house_number = cats_count.house_number
ORDER BY sum_pets DESC
LIMIT 2
; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

### Section III

In [ ]:
c.execute("""SELECT owners_with_dogs.Name AS owner_name
, dogs.Name AS dog_name
, cats.Name AS cat_name
, owners_with_dogs.address
, owners_with_dogs.phone
FROM owners as owners_with_dogs 
JOIN dogs on owners_with_dogs.id = dogs.Owner_ID 
JOIN cats on owners_with_dogs.id = cats.Owner_ID
WHERE address = 'None'
GROUP BY owner_name, address
; """) 
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df